In [40]:
import polars as pl

# Load the bike trips and weather data
trips = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv",
    null_values="NA"
)
weather = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv",
    null_values=["NA", ""]
)

# Ensure date is typed correctly
trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date"))
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date"))

print(trips.height)
print(weather.height)

78704
733


In [41]:
# --- Q1A: Join trips and weather on date (inner join) and show shape + preview ---
trips_weather = trips.join(weather, on="date", how="inner")  # default suffix is "_right"

print(trips_weather.shape)     # expected: (78704, 35)
print(trips_weather.head(5))   # prints "shape: (5, 35)" and the preview table

(78704, 35)
shape: (5, 35)
┌──────┬──────────┬─────────────┬─────────────┬───┬─────────────┬────────┬────────────┬────────────┐
│ id   ┆ duration ┆ start_date  ┆ start_stati ┆ … ┆ cloud_cover ┆ events ┆ wind_dir_d ┆ zip_code_r │
│ ---  ┆ ---      ┆ ---         ┆ on_name     ┆   ┆ ---         ┆ ---    ┆ egrees     ┆ ight       │
│ i64  ┆ i64      ┆ str         ┆ ---         ┆   ┆ i64         ┆ str    ┆ ---        ┆ ---        │
│      ┆          ┆             ┆ str         ┆   ┆             ┆        ┆ i64        ┆ i64        │
╞══════╪══════════╪═════════════╪═════════════╪═══╪═════════════╪════════╪════════════╪════════════╡
│ 4721 ┆ 3        ┆ 2013-08-29T ┆ Market at   ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 20:27:00Z   ┆ 10th        ┆   ┆             ┆        ┆            ┆            │
│ 4812 ┆ 3        ┆ 2013-08-29T ┆ 2nd at      ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 21:30:00Z   ┆ Folsom      ┆   ┆             

In [42]:
# --- Q1B: Average duration for trips on days with ANY weather event ---
avg_bad_weather = (
    trips_weather
    .filter(pl.col("events").is_not_null())
    .select(pl.col("duration").mean().alias("avg_duration"))
)

print(avg_bad_weather)   # one row, column avg_duration


shape: (1, 1)
┌──────────────┐
│ avg_duration │
│ ---          │
│ f64          │
╞══════════════╡
│ 10.34899     │
└──────────────┘


In [43]:
import polars as pl

# Load datasets
Batting = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv",
    null_values="NA"
)
People = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv",
    null_values="NA"
)

# Q2A: Join Batting ⟵ People on playerID (keep all batting rows)
stats_w_bio = Batting.join(People, on="playerID", how="left")
print(stats_w_bio.shape)     # e.g., (112184, 47)
print(stats_w_bio.head(5))   # preview like your screenshot


(112184, 47)
shape: (5, 47)
┌───────────┬────────┬───────┬────────┬───┬──────────┬───────────┬────────────┬────────────┐
│ playerID  ┆ yearID ┆ stint ┆ teamID ┆ … ┆ retroID  ┆ bbrefID   ┆ deathDate  ┆ birthDate  │
│ ---       ┆ ---    ┆ ---   ┆ ---    ┆   ┆ ---      ┆ ---       ┆ ---        ┆ ---        │
│ str       ┆ i64    ┆ i64   ┆ str    ┆   ┆ str      ┆ str       ┆ str        ┆ str        │
╞═══════════╪════════╪═══════╪════════╪═══╪══════════╪═══════════╪════════════╪════════════╡
│ abercda01 ┆ 1871   ┆ 1     ┆ TRO    ┆ … ┆ aberd101 ┆ abercda01 ┆ 1939-11-11 ┆ 1850-01-02 │
│ addybo01  ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ addyb101 ┆ addybo01  ┆ 1910-04-09 ┆ null       │
│ allisar01 ┆ 1871   ┆ 1     ┆ CL1    ┆ … ┆ allia101 ┆ allisar01 ┆ 1916-02-25 ┆ 1849-01-29 │
│ allisdo01 ┆ 1871   ┆ 1     ┆ WS3    ┆ … ┆ allid101 ┆ allisdo01 ┆ 1916-12-19 ┆ 1846-07-12 │
│ ansonca01 ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ ansoc101 ┆ ansonca01 ┆ 1922-04-14 ┆ 1852-04-17 │
└───────────┴────────┴───────┴────────┴───

In [44]:
# Q2B: Total HR by players born in Florida
florida_hr = (
    stats_w_bio
    .filter(pl.col("birthState") == "FL")
    .select(pl.col("HR").cast(pl.Int64).fill_null(0).sum().alias("total_hr"))
)

print(florida_hr)  # one row: total_hr

shape: (1, 1)
┌──────────┐
│ total_hr │
│ ---      │
│ i64      │
╞══════════╡
│ 16225    │
└──────────┘


In [45]:
# --- Q3A (minimal): unique inducted Player IDs only ---
import polars as pl

HallOfFame = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv",
    null_values="NA"
)

hall_inducted = (
    HallOfFame
    .filter((pl.col("inducted") == "Y") & (pl.col("category") == "Player"))
    .select("playerID")
    .unique()
)

print(hall_inducted.shape)
print(hall_inducted.head(5))

(270, 1)
shape: (5, 1)
┌───────────┐
│ playerID  │
│ ---       │
│ str       │
╞═══════════╡
│ kalinal01 │
│ nichoki01 │
│ smithhi99 │
│ colliji01 │
│ thomafr04 │
└───────────┘


In [46]:
# --- Q3B: Average number of career Hits (H) across Hall of Famers ---
hof_hits = (
    Batting
    .join(hall_inducted, on="playerID", how="inner")     # only HOF players
    .group_by("playerID")
    .agg(pl.col("H").fill_null(0).sum().alias("career_hits"))  # total hits per player
    .select(pl.col("career_hits").mean().alias("average_hits")) # one-row result
)

print(hof_hits)  # DataFrame with one row/column: average_hits

shape: (1, 1)
┌──────────────┐
│ average_hits │
│ ---          │
│ f64          │
╞══════════════╡
│ 1717.29918   │
└──────────────┘
